In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from allinput import main_input
from irc6_2007 import *

from reaction import find_bm, find_sf, find_ra, find_rb
from bridge_specs import box, bearing

#### inputs

In [15]:
span = 60

#### defining loads

In [16]:
vehicles = [ll_A, ll_70R, ll_70RT]
classA_pair, class70R, class70RT = [list(i.loadpair) for i in vehicles]
loads = [classA_pair, class70R, class70RT]

#### maxBM, maxSF at equal intervals
when each vehicle from the list of vehicles travels along the span, max reactions (BM and SF+/-) at equally divided intervals are calculated and stored in maxBMs, maxSFs_plus, maxSFs_minus.

In [17]:
#input
def max_rxn(loads, span, detail = 9):
    """

    Args:
        loads: loads pair [(x, axle_load)]
        span: span in metres

    Returns:
        dataframe containing max reactions at intermediate spans for each live load type
    """
    maxBMs = []
    maxSFs_plus = []
    maxSFs_minus = []
    for i in range(len(loads)):
        maxBM = []
        maxSF_plus = []
        maxSF_minus = []
        for j in range(detail):
            at = span / (detail-1) * j
            first_wheel_at = 0
            step = 0.1

            BM = find_bm(span, 0, at)
            SF_plus = find_sf(span, 0, at)
            SF_minus = find_sf(span, 0, at)
            for k in range(int((span + loads[i][-1][0]) / step) + 2):  # '+2' to make sure the load moves all the way to
                # end until it has no effect
                bm = 0
                sf = 0
                for this in loads[i]:
                    a, load = this
                    pos = -a + first_wheel_at
                    bm = bm + find_bm(span, pos, at) * load
                    sf = sf + find_sf(span, pos, at) * load
                first_wheel_at += step
                BM = bm if bm > BM else BM
                SF_plus = sf if sf > SF_plus else SF_plus
                SF_minus = sf if sf < SF_minus else SF_minus
            maxBM.append(round(BM, 3))
            maxSF_plus.append(round(SF_plus, 3))
            maxSF_minus.append(round(SF_minus, 3))
        maxBMs.append(maxBM)
        maxSFs_plus.append(maxSF_plus)
        maxSFs_minus.append(maxSF_minus)
    
    A = ['MaxBM', 'MaxSF+', 'MaxSF-']
    B = ['ClassA', 'Class70RW', 'Class70RT']

    iterables = [A, B]
    index = pd.MultiIndex.from_product(iterables)

    C = []

    for i in [maxBMs, maxSFs_plus, maxSFs_minus]:
        C.extend(i)

    df = pd.DataFrame(C, index=index, columns=[span / (detail -1) * i for i in range(detail)])
#     print(df.loc[('ClassA', 'MaxSF-')])   ## you can navigate using loc, iloc

    new_row = df.loc['MaxSF+'].where(df.loc['MaxSF+'] > abs(df.loc['MaxSF-']), abs(df.loc['MaxSF-']))

    new_row.index = pd.MultiIndex.from_product([['MaxSF'], B])
    df = pd.concat([df, new_row])
    
    return df

#### impact factor

In [18]:
IF = [impact(i.name, span) for i in vehicles]
arr_if = np.array(IF)
arr_with_if = np.array(IF)+1 #including impact factor

distribution factor

In [19]:
dfe = pd.read_excel('outputs/max_e.xlsx', index_col=0)

ecc = dfe['MaxEccentricity'] #eccentricity

def df(ecc, webcc):
    df = (webcc/2+ecc)/webcc
    return round(df, 3)

webcc = 3 #input web center/center distance
distf = [df(i, webcc) for i in ecc]

#### output #1

In [20]:
df_maxR = max_rxn(loads, span, detail = 9) # without impact factor
df_maxR.to_excel('outputs/loads.xlsx') # uncomment to save as excel file
df_maxR.T

MaxBM                         MaxSF+                       MaxSF-  \
        ClassA  Class70RW  Class70RT   ClassA Class70RW Class70RT   ClassA   
0.0      0.000      0.000      0.000  463.432   912.937   672.992    0.000   
7.5   2967.038   5922.025   4417.237  395.105   789.603   585.492  -16.135   
15.0  5106.825  10075.950   7572.175  325.855   664.603   497.992  -59.772   
22.5  6419.363  12527.975   9463.363  256.605   539.603   410.492 -122.885   
30.0  6911.150  13377.400  10092.050  187.355   414.603   322.992 -192.135   
37.5  6640.462  12509.725   9463.588  118.105   289.603   235.492 -261.385   
45.0  5366.425   9928.750   7572.925   51.780   164.603   147.992 -330.635   
52.5  3101.337   5703.475   4416.313   15.300    52.190    60.492 -399.885   
60.0     0.000      0.000      0.000   -0.000    -0.000    -0.000 -469.135   

                            MaxSF                      
     Class70RW Class70RT   ClassA Class70RW Class70RT  
0.0      0.000     0.000  463.432   912.937   672.992  
7.5    -21.120   -60.842  395.105   789.603   585.492  
15.0  -112.063  -148.342  325.855   664.603   497.992  
22.5  -235.397  -234.675  256.605   539.603   410.492  
30.0  -360.397  -322.175  192.135   414.603   322.992  
37.5  -485.397  -409.675  261.385   485.397   409.675  
45.0  -610.397  -497.175  330.635   610.397   497.175  
52.5  -735.397  -584.675  399.885   735.397   584.675  
60.0  -860.397  -672.175  469.135   860.397   672.175